# Machine learning avec les modules Python tensorflow2/keras 

Les concepts de base du *Machine Learning* (ML) peuvent être consultés si besoin dans le notebook du TP1 :  `TP1_MNIST_dense.ipynb`.

## Entraînement / exploitation d'un réseau de neurones convolutionnel pour la reconnaissance de chiffres manuscrits

L'objectif du TP2 est de comprendre le fonctionnement d'un réseau de neurones convolutionnel et de mettre en oeuvre sa construction à l'aide du module **keras**, utilisé comme interface *high level* du module **tensorflow**. <br />

Le séquencement reste le même que celui du TP1 :
- Rappels sur le fonctionnement des réseaux convolutionels
- Chargement des images depuis la banque MNIST.<br>
Dans ce TP les images ne sont pas applaties en vecteurs car un réseau convolutionnel permet de garder la strcuture 2D des images.
- Construction du modèle avec *keras*.
- *one-hot encoding* des labels des images pour les rendre compatibles avec les sorties du réseau de neurones.
- Entraînement & analyse des résultats.

# A/ Les Réseaux de Neurones Convolutionnels (RNC)

## Principes généraux

Les Réseaux de Neurones Convolutionnels (RNC, en anglais *CNN* :*Convolutionnal Neural Network*) proposent des structures particulièrement efficaces pour l'analyse du contenu des images. Pour cela les RNC mettent en oeuvre des traitements et une architecture et bien spécifiques :
- l'extraction des caractéristiques des images (*features*) à l'aide de filtres convolutifs,
- la réduction de la quantité d'information générée par la convolution avec des filtres de *pooling*,
- une architecture qui empile des couches "convolution > activation > pooling..." chargées d'extraire les caractéristiques de l'image (*features*) qui sont au final applaties et envoyées en entrée d'un réseau dense chargé de l'étape de classification.

Dans la suite du TP, nous construirons un RNC inspiré du réseau `LeNet5`, un des premiers RNC proposé par Yann LeCun *et al.* dans les années 90 pour la reconnaisannce des images MNIST :

<p style="text-align:center; font-style:italic; font-size:12px;">
    <img src="img/LeNet5.png" ><br>
    [Lecun, Y.; Bottou, L.; Bengio, Y.; Haffner, P. (1998). "Gradient-based learning applied to document recognition". Proceedings of the IEEE. 86 (11): 2278–2324. doi:10.1109/5.726791.]
</p>

### Extraction des caractéristiques d'une image avec un filtre de convolution

La convolution d'une image par un filtre (aussi appelé noyau, *kernel*) revient à déplacer une _petite fenêtre 2D_ ( 3x3, 5x5 ....) sur l'image et à calculer à chaque fois le produit tensoriel contracté entre les élements du filtre et les pixels de l'image délimités par le filtre (somme des produits terme à terme).<br>

L'animation ci-dessous illustre la convolution d'une image 5x5 par un filtre 3x3 sans *padding* sur les bords : on obtient une nouvelle image plus petite de 3x3 pixels<br>
<p style="text-align:center; font-style:italic; font-size:12px;">
    <img src="img/filter_3x3.png" width="80" style="display:inline-block;">
    <img src="img/Convolution_schematic.gif" width="300" style="display:inline-block;"><br>
    [crédit image : <a href="http://deeplearning.stanford.edu/tutorial">Stanford deep learning tutorial</A>]
</p>

Pour conserver la taille de l'image d'entrée, on peut faire appel à une technique de *padding* pour créer de nouvelles données sur les bords de l'image (par dupplication des données sur les bords, ou ajout de lignes et colonnes de 0... par exemple) : 

<p style="text-align:center; font-style:italic; font-size:12px;">
    <img src="img/padding.gif" width="350"><br>
    [crédit image : <a href="https://towardsdatascience.com/applied-deep-learning-part-4-convolutional-neural-networks-584bc134c1e2"> Arden Dertat</a> ]
</p>

Le but de la convolution est d'extraire des caractéristiques particulières présentes dans l'image source : on parle de "carte des caratéristiques" (*feature map*) pour désigner l'image produite par l'opération de convolution. L'état de l'art conduit à utiliser plusieurs filtres convolutifs pour extraire des caractéristiques différentes : on peut avoir jusqu'à plusieurs dizaines de filtres convolutifs dans un même couche du réseau qui génèrent autant de _feature map_, d'où l'augmentation des données crées par ces opérations de convolution...

#### Exemples d'extraction de caractéristiques avec des filtres convolutifs connus (filtre de [Prewitt](https://fr.wikipedia.org/wiki/Filtre_de_Prewitt)):

À titre d'exemple, la figure ci-dessous montre les *features maps* obtenues en convoluant une image MNIST (un chiffre 7) avec 4 filtres 3x3 connus en traitement d'image (filtres de Prewitt pour l'extraction de contours):

<p style="text-align:center; font-style:italic; font-size:12px;">
    <img src="img/7_mnist_4_filtres.png" width="500"><br>
    [crédit image : JLC]
</p>

On voit que ces filtres agissent comme des filtres de détection de contour : dans les images de sortie, les pixels les plus blancs constituent ce que les filtres ont détecté :
- les filtre (a) et (c) détectent des contours horizontaux inférieurs et supérieurs,
- les filtre (b) et (d) détectent des contours verticaux droite et gauche.

Ces exemples très simples permettent de comprendre comment fonctionne l'extraction des *features* d'une image par filtrage convolutif.

###### Cas général : images RGB traitée par plusieurs filtres de convolution

Dans le cas général où les images correpondent à des tableaux 3D (la troisième dimension étant pour les 3 couleurs R(ed), G(reeen) &  B(lue)), le filtre de convolution est lui aussi un tableau 3D. L'opération reste identique au cas 1D : pour une position du filtre 3D sur l'image, le produit tensoriel contracté du filtre avec le sous-tableau 3D correspondant dans l'image fournit un nombre scalaire, et le balayage du procédé sur toute l'image donne la matrice des caractéristiques (*feature map*) de l'image. 

Par exemple si l'on utilise 10 filtres de convolution 5x5 (10 tableaux de dimensions (5,5,3)) pour traiter (avec  _padding_) une image RGB de 32x32 pixels (tableau de dimensions (32,32,3), on obtient une *feature maps*  de dimensions (32,32,10), soit 10240 termes alors que l'image source n'en a que 1024 !

<p style="text-align:center; font-style:italic; font-size:12px;">
    <img src="img/conv_3D_10.png" width="350"><br>
    [crédit image : <a href="https://towardsdatascience.com/applied-deep-learning-part-4-convolutional-neural-networks-584bc134c1e2"> Arden Dertat</a> ]
</p>

$\leadsto$ Pour réduire la quantité d'information générée par les filtres de convolution sans perdre trop d'information, la convolution est toujours suivie d'une opération de *pooling*.

#### Du filtre convolutif à la couche de neurones convolutifs

L'intégration du filtrage convolutif dans la structure du réseau de neurones donne l'organisation des calculs  suivante :

- Chaque filtre convolutif posssède les mêmes coefficients pour les 3 couleurs : pour le réseau LeNet5 par exemple, chacun des 6 filtres 5x5x3 de la première couche possède seulement 25 coefficients, identiques pour les couleurs R, G & B.

- Chaque unité (neurone convolutif) d'une *feature map* de la couche C1 reçoit 75 pixels (25 pixels rouges $R_i$, 25 pixels vert $G_i$ et 25 pixels bleus $B_i$) délimités par la position du filtre convolutif dans l'image source.

- Le neurone $k$ d'une *feature map* calcule une sortie $y_k = F_a(\sum_{i=1}^{25}{\omega_i(R_i + G_i + B_i) - b_k})$, où $b_k$ est le biais du neurone $k$ et $F_a$ la fonction d'activation (très souvent `relu`).

- pour les 6 filtres convolutrifs de la couche C1, on a donc  6 x (25 + 1) paramètres, soit 156 paramètres inconnues pour cette couche qui seront déterminé par entraînement du réseau.

Le même schéma est utilisé dans toutes les couches convoltionnelles.


### Le *pooling*

Le *pooling* vise à réduire quantité de données à traiter. Comme pour l'opération de convolution, on déplace un filtre sur les éléments du tableau *feature map*  et à chaque position du filtre sur le tableau, on calcule un nombre représentant tous les éléments sélectionnés dans le filtre (par exemple la valeur maximale, ou la moyenne....). Mais contrairement à la convolution, on déplace le filtre sans recouvrement.<br>
Dans l'exemple simplifié ci-dessous, le filtre *max spool* transforme la matrice 8x8 en une matrice 4x4 qui décrit "à peu près" la même information mais avec moins de données :
<p style="text-align:center; font-style:italic; font-size:12px;">
    <img src="img/max_pool_2x2.png" width="350"><br>
    [crédit image : JLC</a> ]
</p>

# C/ Travail à faire

<div class="alert alert-block alert-danger">
<span style="color:brown;font-family:arial;font-size:normal"> 
L'état de l'art actuel des projets de machine learning sous Python préconise l'utilisation d'un <span style="font-weight:bold;">environnement virtuel Python</span> qui permet de maîtriser pour chaque projet les versions de l'interpréteur et des modules Python "sensibles" (comme tensorflow par exemple).

Dans le cas d'un démarrage de l'ordinateur avec une clef USB Ubuntu, on peut considérer que la clef fournit un environnement Python dédié, à condition de ne pas faire de mises à jour des paquets Python avec <span style="font-style:italic">pip install...</span>
    
Dans le cas contraire, la <A href="https://learn.e.ros4.pro/fr/faq/python3/venv/">FAQ Python</A> explique comment créer et utiliser un EVP basé sur **miniconda3** pour utiliser numpy et tensorflow2 avec la bibliothèque optimisée <A href="https://software.intel.com/content/www/us/en/develop/tools/oneapi/components/onemkl.html">MKL</A>.
</span>
</div>

### Modules Python tensorflow/keras

Le module **keras** qui permet une manipulation de haut niveau des objets **tensorflow** est intégré dans le module **tensorflow** (tf) depuis la version 2. <br>
La documentation du module **tf.keras** à consulter pour ce TP est ici : https://www.tensorflow.org/api_docs/python/tf/keras. 

Versions des modules Python validées pour ce TP sous Ubuntu 20 / Python3.8.5 :
- tensorflow 2.4.0 incluant tensorflow.keras 2.4.0
- OpenCV >= 4.2.0

In [ ]:
import tensorflow as tf
from tensorflow import keras
import sys, cv2
print(f"Python    : {sys.version.split()[0]}")
print(f"tensorflow: {tf.__version__} incluant keras {keras.__version__}")
print(f"OpenCV    : {cv2.__version__}")

#### Incrustation des tracés matplotlib dans le cahier IPython et import de modules utiles :

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

#### Affectation des graines (*seed*) des générateurs pseudo-aléatoires :

Le TP1 donne plus de détails sur la reproductibilité des générateurs pseudo-aléatoires utilisés.

In [ ]:
SEED = 123
np.random.seed(SEED)      
tf.random.set_seed(1234)

## 1 - Récupération et mise en forme des données MNIST

Consulter la documentation de la fonction `load_data` sur la page [tf.keras.datasets.mnist.load_data](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data) puis compléter la cellule ci-dessous pour charger les données du MNIST en nommant les données renvoyées :<br>
- `im_train`, `im_test` pour les images d'entraînement et de test,
- `lab_train`, `lab_test` pour les labels des images d'entraînement et de test.

(En cas de message d'erreur de type _"SSL error...."_ pour téléchager les données du MNIST, voir [Python SSL Certification Problems in Tensorflow](https://stackoverflow.com/questions/46858630/python-ssl-certification-problems-in-tensorflow))

La cellule ci-dessous affiche les attributs `shape` et `dtype` des tableaux numpy obtenus : les valeurs son-elles cohérentes ?

In [ ]:
print("im_train -> shape:", im_train.shape, ", dtype:", im_train.dtype,)
print("im_test  -> shape:", im_test.shape,  ", dtype:", im_test.dtype,)
print("lab_train-> shape:", lab_train.shape,  ", dtype:", lab_train.dtype)
print("lab_test -> shape:", lab_test.shape,  ", dtype:", lab_test.dtype)

## Mise en forme des données d'entrée

Les couches convolutionnelles du module keras attendent par défaut des tableaux multidimentionnels de la forme `(batch_size, height, width, depth)` :
- `batch_size` : nombre d'image en entrée,
- `height` et `width` : hauteur et largeur des images (en pixels),
- `depth` : profondeur des tableaux (`3` pour une image RGB, `1` pour une image en ton de gris).

La forme des images MNIST est :

In [ ]:
im_train.shape, im_test.shape

Il faut donc rajouter une dimension (égale à `1`) après la troisième dimension `28`, par exemple avec la méthode `reshape` des tableaux `ndarray` de numpy.

Compléter la cellule suivante pour définir `x_train` et `x_test` obtenus :
- en ajoutant une quatrième dimension égale à 1 aux tableaux `im_train` et `im_test` 
- et en normalisant les valeurs.

Vérification :

In [ ]:
im_train.shape, x_train.shape, im_test.shape, x_test.shape

### Mise au format *one-hot* des labels MNIST

Il faut mettre les labels MNIST au format *one-hot* qui permet de calculer l'erreur entre un label et la sortie de la couche de classification du réseau (voir le notebook `TP1_MNIST_dense.ipynb` pour les détails sur le format *one-hot*).<br>
Consulter la page [tf.keras.utils.to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) sur la fonction `to_categorical` et en déduire comment transformer les tableaux `lab_train` et `lab_test` en tableaux `y_train` et `y_test` contenant des vecteurs encodés *hot-one* :


## 3 - Construction du réseau de neurones

On va maintenant construire dans la cellule ci-dessous le réseau de neurones **convolutionnel** à l'aide du module **keras**.

Comme dans le TP1, on crée un objet `model` instance de la classe `Sequential` (cf [tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)), puis on complète `model` de façon  incrémentale en ajoutant chaque couche avec la méthode `add` :

- La couche d'entrée de type `InputLayer` (cf [tf.keras.layers.InputLayer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/InputLayer)) sert à spécifier la forme des données d'entrée.<br>
La forme attendue par keras pour des images en entrée est (height,widt,depth) : on pourra l'obtenir par exemple avec l'attribut `shape` de n'importe quelle image du tableau `x_train`.<br><br>

- Les couches convolutionnelles sont de type `Conv2D` (cf [tf.keras.layers.Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)) :
    - les 2 premiers arguments (positionnels) sont : 
        - le nombre de filtres de la couche
        - la forme du filtre : on peut spécifier  `N` ou `(N,N)` pour spécifier un filtre N x N
    - les autres arguments (nommés) utilisés sont :
        - `stride` : le pas du déplacement du filtre de convolution, valeur par défaut :  `stride=1` (équivalent à `(1, 1)`))
        - `padding=valid` : pas de padding, ou `padding=same` : sortie de même dimensions que l'entrée (défaut : `valid`)
        - `activation` : choix de la fonction d'activation (`'relu'`, '`tanh'`...)<br><br>
        
- Les couches de *pooling* du réseau LeNet5 historique utilisent un filtre *average pool* qui correspond à la classe `AveragePooling2D`  (cf [tf.keras.layers.AveragePooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/AveragePooling2D)), mais on aura de meilleurs résultats avec un filtrage *max pool* qui retient la valeur max des pixels dans la fenêtre du filtre (voir la page [tf.keras.layers.MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)). Principaux arguments à utiliser avec `MaxPool2D` :
    - `pool_size` :  `N` ou `(N,N)` pour spécifier un filtre N x N (défaut : `(2,2)`)
    - `strides` : int, tuple de 2 int, ou None. Si None (valeur par défaut), prend la même valeur que `pool_size`
    - `padding` : comme pour la classe `Conv2D`<br><br>

- Pour applatir les 16 *feature maps* 5x5 en un vecteur de 16 * 5 * 5 = 635 éléments, on peut utiliser une couche  `Flatten` (cf [tf.keras.layers.Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten))<br><br>

En s'inspirant de la structure du réseau `LeNet5` et des spécification ci-dessus, on obtient :
- couche d'entrée : `Input(shape=x_train[0].shape)` : on utilise l'attribut `shape` de la 1ère image qui vaut (28,28,1).
- couche C1 : `Conv2D(6, 5, padding='same', activation='relu', name='C1')`
- couche S2 : `MaxPool2D(pool_size=2, name='S2')`
- couche C3 : `Conv2D(16, 5, padding='valid', activation='relu', name='C3')`
- couche S4 : `MaxPool2D(pool_size=2, name='S4')`
- couche d'applissement : `Flatten()`
- couche C5 : `Dense(200, activation='relu', name='C5')`
- couche F5 : `Dense(84, activation='relu', name='F6'`
- couche OUTPUT : `Dense(nb_classe, activation='softmax', name='Output')`

Une fois construit, le réseau doit être compilé (au sens de tensorflow) avec la méthode `compile` en utilisant par exemple les :
- `loss='categorical_crossentropy'` : choix de la fonction d'erreur (cf [tf.keras.categorical_crossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/categorical_crossentropy))
- `optimizer='adam'` : choix de l'optimiseur Adam (cf page [tf.keras.optimizers.Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam))
- `metrics=['accuracy']` pour obtenir les données permettant de tracer les courbes de performance.

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten

nb_classe = 10
tf.random.set_seed(SEED)

model = 



Avec la méthode `summary` de l'objet `model`, faire afficher la description du modèle : noter les valeurs des paramètres...

La fonction `tf.keras.utils.plot_model` permet aussi de dessiner la structure du réseau (voir la page [tf.keras.utils.plot_model](https://www.tensorflow.org/api_docs/python/tf/keras/utils/plot_model)).<br>
Faire tracer la structure du modèle en ajoutant l'option `show_shapes=True` à l'appel de `model` :

### Sauvegarde de l'état initial du  réseau

On peut sauvegarder l'état initial des poids du réseau non-entraîné (valeurs aléatoires) avec la méthode `Model.save_weights`. <br>
Ce sera utile plus loin pour remettre le réseau à son état initial avant de relancer d'autres entraînements :

In [ ]:
import os

# vérifier que le dossier 'weights' existe et sinon le créer:
if not os.path.exists("weights"): os.mkdir("weights")

# sauvegarde des poinds du réseau initial:
key = 'conv_initial'
model.save_weights('weights/'+key)

# afficher les fichiers créés:
files=[os.path.join("weights",f) for f in os.listdir("weights") if f.startswith(key)]
for f in files: print(f)

Remarque : la méthode `save_weights` utilise la partie `key` du chemin passé en argument pour préfixer les fichiers créés.<br>
Lors de la lecture ultérieure des poids du réseau avec la méthode `load_weights` de la classe `Sequential`, il suffira de donner la même information pour retrouver les bons fichiers.

## 4 - Entraînement du réseau avec test à chaque `epoch`

Consulter au besoin la documentation de la méthode `fit` dans la page [tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential). 

Compléter la cellule ci-dessous pour entraîner le réseau en utilisant la méthode `fit` de l'objet `model` avec les arguments :
- `x_train` : les 60000 images 
- `y_train` : les 60000 labels encodés *one-hot*.
- `epochs=10` : faire 10 fois l'entraînement complet.
- `batch_size=64` : découper le jeu des données d'entrée (les 60000 images) en "lots" (*batch*) de taille `batch_size`.<br>
La mise à jour des poids du réseau est faite au bout de `batch_size` échantillons d'entrée. La valeur de `batch_size` (par défaut est 32) est un paramètre qui influe beaucoup sur la qualité de l'apprentissage : on peut essayer d'autres valeurs (64, 128 ...) et observer comment évoluent les performances d'entraînement).

Pour avoir un indicateur réaliste de la qualité du réseau entraîné on teste à chaque `epoch` la précison du réseau entraîné en utilisant les données de test : il faut passer l'agument `validation_data` à la méthode `fit`, en lui affectant le tuple des données de test `(x_test, y_test)`

In [ ]:
# au cas on on exécute plusieurs fois cette cellule, il faut ré-initialiser 
# les poids du réseau à leur valeur initiale si on veut comparer les entraînements...
key = 'conv_initial'
model.load_weights('weights/'+key)  
tf.random.set_seed(SEED)

hist = model.fit(...)

L'objet `hist` retourné par la méthode `fit` possède un attribut `history` de type dictionnaire dont les clefs `'loss'`, `'accuracy'` contiennent l'évaluation de la fonction de cout et de la précision du réseau à la fin de chaque (*epoch*) avec les données d'entraînement. Les clefs `'val_loss'` et `'val_accuracy'` sont associées aux données de test.

In [ ]:
hist.history.keys()

In [ ]:
print(hist.history['loss'])
print(hist.history['accuracy'])
print(hist.history['val_loss'])
print(hist.history['val_accuracy'])

#### Tracé des courbes `accuracy` et `loss`  de l'entraînement et des test :

La fonction `plot_loss_accuracy` du module `utils.tools` (présent dans le répertoire du notebook) permet de tracer les courbes de précision et de perte en utilisant les données stockées dans l'objet `hist`. Faire tracer ces courbes :

### Arrêter automatiquement l'entraînement avant *over-fit*

Keras propose des outils pour arrêter automatiquement l'apprentissage en surveillant par exemple la croissance de `val_accuracy` ou la décroisance de `val_losss` d'une `epoch` à l'autre (cf le callback  [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping)).

On peut ainsi définir une liste de fonctions *callback* que l'on peut passer en argument à la fonction `fit` avec l'agument nommé  `callbacks` :

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

callbacks_list = [
    EarlyStopping(monitor='val_loss',  # la grandeur à surveiller
                  patience=2,              # on accepte que 'val_accuracy' puisse diminuer 2 fois de suite
                  restore_best_weights=True,
                  verbose=1)
]

# recharger l'état initial du réseau:
key = 'conv_initial'
model.load_weights('weights/'+key)  
tf.random.set_seed(SEED)

hist = model.fit(x_train, y_train,
                    validation_data=(x_test, y_test),
                    epochs=15, 
                    batch_size=12, 
                    callbacks = callbacks_list)

Tracer les courbes `loss` et `accuracy` :

In [ ]:
from utils.tools import plot_loss_accuracy
plot_loss_accuracy(hist)

Le réseau convolutionnel tend vers une meilleure précision voisine de 99%.

### Sauvegarder les poids du  réseau entraîné

La méthode `save_weights` de la classe `Sequential`permet d'enregistrer les **poids** du réseau entraïné dans un fichier :

In [ ]:
import os
# vérifier que le dossier 'weights' existe et sinon le créer:
if not os.path.exists("weights"): os.mkdir("weights")

# sauvegarde des poids du réseau entrainé:
key = 'conv_trained'
model.save_weights('weights/'+key)

# afficher les fichiers créés:
files=[os.path.join("weights",f) for f in os.listdir("weights") if f.startswith(key)]
for f in files: print(f)

### Sauvegarder la structure du réseau et ses poids

La méthode `save` de la classe `Sequential` permet d'enregistrer **toute la structure et les poids** du réseau entraïné dans un fichier.<br />
Ceci permet de recréer plus tard *from scratch* le réseau entrainé pour passer en phase exploitation du réseau par exemple, en utilisant la fonction`tf.keras.models.load_model` :

In [ ]:
import os
# vérifier que le dossier 'weights' existe et sinon le créer:
if not os.path.exists("models"): os.mkdir("models")

# sauvegarder structure réseau + poids :
key = 'conv_trained'
model.save('models/'+key) 

# afficher les fichiers créés:
files=[os.path.join("models",f) for f in os.listdir("models") if f.startswith(key)]
for f in files: print(f)

## 6/ Exploitation du réseau avec le jeu de test

La méthode `predict` de l'objet `model` permet de calculer les inférences du réseau pour une ou plusieurs entrées (voir la méthode `predict`dans la page [tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)).

La cellule ci-dessous montre la mise en oeuvre de la méthode `predict`, et comment exploiter la représentation  *one-hot* renvoyée par `fit` en utilisant la méthode `argmax` des tableaux de numpy :

In [ ]:
from utils.tools import plot_images

i = 100   # numéro image de test
rep = model.predict(x_test[i:i+1]) # Attention: x doit être un tableau de matrices...
                                   # => x[i] ne convient pas !

print(f"sortie du réseau pour l'image de rang {i} :\n{rep[0]}")

# limiter l'affichage des composantes des tableaux numpy à 1 chiffre :    
with np.printoptions(formatter={'float':'{:.2f}'.format}):    
    print(f"\nsortie réseau arrondie à 2 chiffre : {rep[0]}")
    
print(f"rep[0].argmax() donne : {rep[0].argmax()}")

print(f"\nLa bonne réponse est {lab_test[i]} soit en 'one-hot' : {y_test[i]}")

plot_images(im_test,i,1,1)

#### Utilité de la méthode numpy `ndarray.argmax` pour décoder le tableau de vecteurs *one-hot* renvoyé par la méthode `predict`

Quand on calcule l'inférence du réseau `model` avec les données de test par exemple, on obtient un résultat qui est un tableau de vecteurs codés *one-hot*, comme le détaille la cellue suivante :

In [ ]:
results = model.predict(x_test)

print("forme du tableau 'results':", results.shape)
print("allure des vecteurs du tableau 'result', par exemple :")
with np.printoptions(formatter={'float':'{:.4f}'.format}): 
    print("results[0]  :", results[0])
    print("results[-1] :", results[-1])

En écrivant `results.argmax(axe=-1)`, on obtient le tableau des `argmax` de chaque vecteur -> c'est directement le tableau des chiffres reconnus par le réseau :

In [ ]:
chiffres_reconnus = results.argmax(axis=-1)

print("chiffres_reconnus -> shape:", chiffres_reconnus.shape, ", dtype:", chiffres_reconnus.dtype)
print(f"contenu de chiffres_reconnus : {chiffres_reconnus}")

Retrouver le taux de réussite du réseau entrainé en utilisant les inférences du réseau pour les entrées `x_test` et les labels `lab_test` :

In [ ]:
results = model.predict(x_test)
chiffres_reconnus = results.argmax(axis=-1)

success = 0
for chiffre, label in zip(chiffres_reconnus, lab_test):
    success += (chiffre == label)
print(f"taux de réussite : {success/len(x_test)*100:.2f} %")

###  Afficher la matrice de confusion

La cellule suivante définie la fonction `show_cm_mnist` qui affiche la **matrice de confusion**.

La matrice de confusion permet de visualiser :
- sur la diagonale : les bonnes réponses du réseau, avec dans chaque case le nombre de bonnes réponses
- hors diagonale : les erreurs du réseau, avec dans chaque case la fréquence d'apparition de l'erreur.

In [ ]:
import pandas as pd
from seaborn import heatmap
from sklearn.metrics import confusion_matrix

def show_cm_mnist(target, results, classes):
    # target  : the actual labels (one-hot format)
    # results : the labels computed by the trained network (one-hot format)
    # classes : list of possible label values
    predicted = np.argmax(results, axis=-1) # tableau d'entiers entre 0 et 9 
    cm = confusion_matrix(target, predicted)
    df_cm = pd.DataFrame(cm, index=classes, columns=classes)
    plt.figure(figsize=(11,9))
    heatmap(df_cm, annot=True, cbar=False, fmt="3d")
    plt.xlabel('actual label')
    plt.ylabel('predicted label')
    plt.show()

Faire afficher la matrice de confusion en lui passant les labels attendus `lab_test` et les labels calculés par le model :

Effectivement, il y a assez peu d'erreurs hors diagonale.

# D/ Bonus

## Faire reconnaître des images originales à un réseau entraîné avec les images MNIST

Plusieurs possibilités :
- utiliser des images crées pour l'occasion... 
- utiliser les images toutes prêtes du répertoire `chiffres`.

Si tu crées tes propres images de chiffres écrits à la main, il faut :
- les mettre au format MNIST (20x28 pixels en ton de gris, chiffre centré dans l'image) 
- les placer dans un répertoire spécifique ,
- affecter ce nom de ce répertoire à `images_dir` dans la cellule ci-dessous :

In [ ]:
import os

# changer le nom du répertoire au besoin :
images_dir = "chiffres"

images = [os.path.join(images_dir,f) for f in os.listdir(images_dir) if f.endswith(".png")]
images.sort()

print(f"Images du dossier <{images_dir}> à reconnaître :")
for im in images: print(im)

### Lecture des fichiers image avec openCV

Les images doivent être convertie en image en ton de gris de 28 x 28 pixels pour pouvoir être traitées par le réseau entraîné sur les images MNIST.

Plusieurs fonctions du module OpenCV pourront être utilisées :
- `cv2.imread(file_name)` : pour lire un fichier image aux formats standards (PNG, JPG,...)
- `cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)` : pour convertir le tableau `img` renvoyé par `cv2.imread` en tons de gris
- `cv2.resize` : pour retailler l'image.

La cellule ci-dessous montre un exemple de lecture et traitement avec OpenCV des images du dossier `chiffres` qui sont déjà au format 28 x 28 pixe :

In [ ]:
import cv2
my_images = []
for image_path in images:
    img = cv2.imread(image_path)                    # lecture fichier image
    img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # conversion en tons de gris
    my_images.append(img_gray)

Visualisation des images lues :

In [ ]:
plot_images(my_images, 0, 1, 10)

Inversion des images pour avoir des chiffres doivent être en blanc sur fond noir :

In [ ]:
my_images = [255 - im for im in my_images]
plot_images(my_images, 0, 1, 10)

On peut maintenant :
- transformer des matrices 28x28 en tableaux (28,28,1) de float normalisés,
- calculer les inférences du réseau entaîné de votre choix (`model` ou autre...) avec les images perso en entrée,
- faire afficher la précision obtenue et la matric de confusion.

Que suggèrent les  résultats ?

# Autres ressources intéressantes... des vidéos :

In [1]:
%%HTML
<iframe src="https://www.youtube.com/embed/trWrEWfhTVg" width="800" height="450" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [2]:
%%HTML
<iframe src="https://www.youtube.com/embed/aircAruvnKk" width="800" height="450" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [3]:
%%HTML
<iframe src="https://www.youtube.com/embed/IHZwWFHWa-w" width="800" height="450" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [4]:
%%HTML
<iframe src="https://www.youtube.com/embed/Ilg3gGewQ5U" width="800" height="450" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>